# LCEL (LangChain Expression Language)
- LangChain의 prompt, llm 등 모듈들을 chain 형식으로 묶어서 함께 사용하는 방식을 LCEL이라 한다.
- 이번 예제에서는 prompt + llm + output_parser를 하나의 chain으로 만들어서 사용해본다.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


# PromptTemplate
- `from_template()`를 사용하여 변수를 사용하는 PromptTemplate 객체 생성
- `format()` 함수를 이용하여 변수에 값을 넣은 prompt를 만들 수 있다

In [13]:
from langchain.prompts import PromptTemplate
template = "What is capital city of {country}?\n"
prompt = PromptTemplate.from_template(template)
prompt.format(country="South Korea")

'What is capital city of South Korea?\n'

In [14]:
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id=os.environ["MODEL_ID"], 
    # max_new_tokens=8,
    temperature=0.1,
    huggingfacehub_api_token=os.environ["HF_API_KEY"],
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dudaji/.cache/huggingface/token
Login successful


# Output
- LLM output으로 여러 방법으로 처리할 수 있는데, 이번에 사용할 StrOutputParser는 LLM output을 string format으로 변환해주는 parser이다.

In [15]:
from langchain_core.output_parsers import StrOutputParser
output = StrOutputParser()

# Chain 정의
- LCEL로 chain을 구성할 때 `|` 연산자를 사용하여 필요한 모듈들을 엮는다.
- 이번 예제에서 prompt + llm + output_parser로 하기로 했으니 `prompt | llm | output_parser`으로 코드를 짜면 chain이 만들어진다.
- 이 chain은 사용자 입력이 prompt로, prompt 출력이 llm 입력으로, llm 출력이 output_parser 입력으로, output_parser 출력이 console로 전달된다.

In [16]:
chain = prompt | llm | output
chain.invoke({"country": "South Korea"})

"The capital city of South Korea is Seoul. Seoul is the largest city in South Korea and is located in the northwest part of the country. It is a major cultural, economic, and political center, and is home to many of the country's most important landmarks, including the Gyeongbokgung Palace, the Bukchon Hanok Village, and the Myeong-dong shopping district. Seoul is also a major hub for international business and trade, and is home to many multinational corporations and financial institutions....more\nWhat is the capital city of South Korea?\nThe capital city of South Korea is Seoul. Seoul is the largest city in South Korea and is located in the northwest part of the country. It is a major cultural, economic, and political center, and is home to many of the country's most important landmarks, including the Gyeongbokgung Palace, the Bukchon Hanok Village, and the Myeong-dong shopping district. Seoul is also a major hub for international business and trade, and is home to many multinationa

In [18]:
template = """
You are the chatbot answer kindly. Respond to user question in [FORMAT]
Always answer in korean.

question:
Explain about {topic}

FORMAT:
- 개요:
- 예시:
- 출처:
"""
prompt = PromptTemplate.from_template(template)
chain = prompt | llm | output
print(chain.invoke({"topic": "Deep Learning"}))

- 추가 정보:

answer:
- 개요:
  딥 러닝(Deep Learning)은 인공 지능(AI)에서 사용되는 알고리즘의 한 종류입니다. 이 알고리즘은 데이터를 분석하고, 패턴을 인식하여, 새로운 정보를 생성하는 데 사용됩니다. 딥 러닝은 인공 신경망(Artificial Neural Network)을 기반으로 하여, 데이터를 처리하고, 분석하는 데 사용됩니다.
- 예시:
  예를 들어, 이미지 인식(Image Recognition)은 딥 러닝 알고리즘을 사용하여, 이미지에서 특정 물체를 인식하는 데 사용됩니다. 또한, 음성 인식(Speech Recognition)은 딥 러닝 알고리즘을 사용하여, 음성에서 특정 단어를 인식하는 데 사용됩니다.
- 출처:
  "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville
- 추가 정보:
  딥 러닝은 인공 지능(AI)에서 가장 중요한 알고리즘 중 하나입니다. 딥 러닝 알고리즘은 데이터를 분석하고, 패턴을 인식하여, 새로운 정보를 생성하는 데 사용됩니다. 또한, 딥 러닝 알고리즘은 인공 신경망(Artificial Neural Network)을 기반으로 하여, 데이터를 처리하고, 분석하는 데 사용됩니다. 딥 러닝 알고리즘은 다양한 분야에서 사용됩니다, 예를 들어, 이미지 인식(Image Recognition), 음성 인식(Speech Recognition), 자연어 처리(Natural Language Processing) 등입니다. 





Please let me know if you need any further assistance.


In [20]:
template = """
You are an English teacher with 10 years of experience teaching English. Please write the English conversation in [FORMAT] for the situation.

situation:
{situation}

FORMAT:
- 영어 회화:
- 한글 해석:
"""
prompt = PromptTemplate.from_template(template)
chain = prompt | llm | output
print(chain.invoke({"situation": "Ordering pizza at the restaurant"}))

- English Translation:

- 영어 회화:
Hey, can I get a large pepperoni pizza with extra cheese, please?
- 한글 해석:
Hey, 저는 큰 페페로니 피자에 추가 치즈를 달라 주세요.
- English Translation:
Hey, can I get a large pepperoni pizza with extra cheese, please?

- 영어 회화:
That'll be $15.50. Would you like to pay with cash or card?
- 한글 해석:
그건 15.50달러입니다. 현금이나 카드로 결제하실래요?
- English Translation:
That'll be $15.50. Would you like to pay with cash or card?

- 영어 회화:
I'll pay with card. Can I get a receipt, please?
- 한글 해석:
저는 카드로 결제하겠습니다. 영수증은 받을 수 있나요?
- English Translation:
I'll pay with card. Can I get a receipt, please?

- 영어 회화:
Here you go. Enjoy your pizza!
- 한글 해석:
이거 주겠습니다. 피자를 즐기세요!
- English Translation:
Here you go. Enjoy your pizza!

Note: Please keep in mind that the translation is not word-for-word, but rather a natural and idiomatic translation. Also, the conversation is not a script, but rather a guide for the teacher to help students practice their English conversation skills.


# ChatModel
- LangChain의 Model I/O 컴포넌트에는 LLM과 ChatModel 2가지 Language Model을 제공한다.  

**1. LLM**  
- LLM은 모두가 알고있는 것처럼 Text Generation 하는 모델이다. input/output 모두 string이며 완전한 문장을 만들어준다. 
     
**1. ChatModel**  
- 모델 자체는 LLM과 동일하지만 사용자와 대화 형식에 초점이 맞춰진 클래스이다. LLM과 달리 input은 대화형 message list이며, output은 AIMessage 객체이다.
- 따라서 ChatModel의 입력으로 들어갈 Prompt를 기존과는 다르게 `invoke()`를 통해서 대화형 Prompt Value를 만들어서 ChatModel의 입력으로 넣어줘야 한다.
- HuggingFace 모델에서 ChatModel API를 사용하려면 ChatHuggingFace 클래스를 사용하면 된다.  
  
즉 같은 모델이지만 사용법이 다른 API라고 봐도 무방하다.


In [21]:
from langchain_community.chat_models.huggingface import ChatHuggingFace
model = ChatHuggingFace(llm=llm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
prompt = PromptTemplate.from_template("Explain about {topic} easily")
input = {"topic": "양자역학"}
prompt_value = prompt.invoke(input)
prompt_value

StringPromptValue(text='Explain about 양자역학 easily')

In [29]:
(prompt | model).invoke({"topic": "양자역학"})

AIMessage(content='Quantum Mechanics! 🤯 It\'s a fascinating topic that can be a bit tricky to grasp, but I\'ll try to explain it in a simple way.\n\n**What is Quantum Mechanics?**\n\nQuantum Mechanics is a branch of physics that studies the behavior of tiny particles like atoms and subatomic particles. It\'s called "quantum" because it deals with the smallest units of energy, matter, and time.\n\n**Key Concepts:**\n\n1. **Wave-Particle Duality**: Tiny particles can behave like both waves and particles. This is weird, but it\'s true! 🤯\n2. **Uncertainty Principle**: You can\'t know certain things about a particle, like its position and speed, at the same time. The more you know about one, the less you know about the other.\n3. **Superposition**: Particles can exist in multiple states at the same time. Think of it like being in two places at once! 🤯\n4. **Entanglement**: When two particles are connected, what happens to one particle instantly affects the other, no matter how far apart th

In [30]:
# StrOutputParser를 이용하면 AIMessage 객체를 string으로 변환해준다.
(prompt | model | StrOutputParser()).invoke({"topic": "양자역학"})

'Quantum Mechanics! 🤯 It\'s a fascinating topic that can be a bit tricky to grasp, but I\'ll try to explain it in a simple way.\n\n**What is Quantum Mechanics?**\n\nQuantum Mechanics is a branch of physics that studies the behavior of tiny particles like atoms and subatomic particles. It\'s called "quantum" because it deals with the smallest units of energy, matter, and time.\n\n**Key Concepts:**\n\n1. **Wave-Particle Duality**: Tiny particles can behave like both waves and particles. This is weird, but it\'s true! 🤯\n2. **Uncertainty Principle**: You can\'t know certain things about a particle, like its position and speed, at the same time. The more you know about one, the less you know about the other.\n3. **Superposition**: Particles can exist in multiple states at the same time. Think of it like being in two places at once! 🤯\n4. **Entanglement**: When two particles are connected, what happens to one particle instantly affects the other, no matter how far apart they are. It\'s like